## Connessione con VLC

Per utilizzare il programma come player, segui questi passaggi:

1. **Scaricare VLC**  
   - Scarica e installa VLC dal sito ufficiale.

2. **Aprire VLC**  
   - Avvia l'applicazione VLC sul tuo dispositivo.

3. **Accedere alle impostazioni avanzate**  
   - Vai su:  
   `Strumenti > Preferenze`  
   - Clicca su `Tutto` (in basso a sinistra).

4. **Abilita l'interfaccia web**  
   - Nella lista a sinistra, seleziona `Interfacce principali`.  
   - Sulla destra, spunta la voce `Web`.

5. **Configura Lua e la password**  
   - Espandi la sezione `> Interfacce principali` nella lista a sinistra.  
   - Seleziona `Lua`.  
   - In `HTTP Lua` (lato destro), inserisci la password: `123`

6. **Salva e riavvia**  
   - Clicca `Salva` in basso a destra.  
   - Chiudi e riapri VLC.  
   *Assicurati che VLC rimanga aperto in background.*

7. **Accedi al player web**  
   - Apri il browser e vai all'indirizzo:  [http://localhost:8080](http://localhost:8080)

8. **Aggiungi contenuti**  
   - Scarica file multimediali (es. canzoni e sound effects) sul tuo dispositivo.

9. **Modifica i path**  
   - Aggiorna il `path` nella sezione del codice contrassegnata con *VLC functions* che indica le canzoni da gestire
   - Aggiorna il `path_start_sound` e `path_completed_sound` nella sezione del codice contrassegnata con *#Er codice* che indicano i sound effects al rilevamento di gesti

10. **Modifica il nome delle delle canzoni nei file di codice**  
   - Aggiorna il `enqueue_tracks` nella sezione del codice contrassegnata con *VLC functions*

In [1]:
# # pip install


# !pip install requests
# !pip install cv2
# !pip install mediapipe
# !pip install pygame

In [2]:
#import


import cv2
import mediapipe as mp
import time
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import quote
from requests.auth import HTTPBasicAuth
import math
from playsound import playsound
import threading
import os
import pygame
import numpy as np


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### VLC


Assicurati di aggiornare la variabile `path` con il tuo percorso di salvataggio delle canzoni.

Personalizza i nomi delle canzoni nelle funzioni `enqueue_track` per aggiungere i tuoi brani preferiti alla playlist.

In [3]:
#VLC connection


VLC_PASSWORD = '123'
VLC_URL = "http://localhost:8080/requests/status.xml"

def vlc_command(command):
    response = requests.get(f"{VLC_URL}?command={command}", auth=HTTPBasicAuth('', VLC_PASSWORD))
    if response.status_code == 200:
        print(f"[✓] Comando '{command}' eseguito")
    else:
        print(f"[✗] Errore: {response.status_code}")




#VLC functons
def enqueue_track(filepath_or_url):
    # Se è un file locale, aggiungiamo file:///
    if filepath_or_url.startswith("http"):
        input_url = filepath_or_url
    else:
        input_url = "file:///" + quote(filepath_or_url.replace("\\", "/"))

    full_url = f"{VLC_URL}?command=in_enqueue&input={input_url}"
    response = requests.get(full_url, auth=HTTPBasicAuth('', VLC_PASSWORD))


# Funzione per convertire un percorso in URI valido per VLC
def get_vlc_uri(file_path):
    # Sostituisci \ con / e codifica caratteri speciali (spazi, parentesi, etc.)
    uri_path = file_path.replace("\\", "/")
    return urllib.parse.quote(uri_path)




#cambia questa path con la tua
path = r"music"

# Aggiungi alcune canzoni all'interno della path precedente per iniziare
enqueue_track(path + r"\Arctic Monkeys - Knee Socks.mp3")
enqueue_track(path + r"\Future-Mask Off (Marimba Ringtone).mp3")

# Funzioni per i comandi VLC
def play_pause():
    vlc_command("pl_pause")

def next():
    vlc_command("pl_next")

def prev():
    vlc_command("pl_previous")

def vol_up():
    vlc_command("volume&val=+80")

def vol_dw():
    vlc_command("volume&val=-80")
    
# def stop():
#     vlc_command("pl_stop")


    
    


In [4]:
# gesture recognition functions



def check_open_hand(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # IDs delle articolazioni delle dita (MediaPipe landmarks)
    finger_tips = [4, 8, 12, 16, 20]  # pollice, indice, medio, anulare, mignolo
    finger_pips = [2, 6, 10, 14, 18]   # articolazioni corrispondenti

    open_fingers = 0
    
    if hand_landmarks.landmark[0].y > min(all_y[1:]): #se il polso si trova più in basso rispetto a tutte le altre
        for i in range(5):
            tip = hand_landmarks.landmark[finger_tips[i]] #punta del dito corrente
            pip = hand_landmarks.landmark[finger_pips[i]] #articolazione del dito corrente
            
            # Per il pollice controlliamo la coordinata x
            if i == 0:  # pollice
                if (hand_landmarks.landmark[0].x < tip.x and tip.x > pip.x and hand_landmarks.landmark[1].x > hand_landmarks.landmark[0].x ) or \
                    (hand_landmarks.landmark[0].x > tip.x and tip.x < pip.x and hand_landmarks.landmark[1].x < hand_landmarks.landmark[0].x ):
                    open_fingers += 1
            # Per le altre dita controlliamo la coordinata y
            else:
                if tip.y < pip.y and\
                    abs(hand_landmarks.landmark[4].x-hand_landmarks.landmark[20].x)>0.1 and\
                    abs(hand_landmarks.landmark[0].y-hand_landmarks.landmark[12].y)>0.3    :  # la punta è sopra l'articolazione (dito aperto)
                    open_fingers += 1
        
        # Consideriamo la mano aperta se almeno 4 dita sono aperte (puoi cambiare a 5 per essere più preciso)
        return open_fingers >= 5
    else:
        return False
    
    
def check_thumbs_up(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    thumb_y = all_y[3:5]
    
    other_y = all_y[5:] #but not 0
    
    if max(thumb_y) < min(other_y) and abs(hand_landmarks.landmark[4].y-hand_landmarks.landmark[5].y)>0.1 :
        return True
    else:
        return False
    

def check_thumbs_down(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_y = all_y[3:5] #joints 3,4
    other_y = all_y[5:] #all joints but 0,2,3,4
        

    if min(thumb_y) > max(other_y) and\
        abs(hand_landmarks.landmark[4].x - hand_landmarks.landmark[17].x) < 0.2 and\
        abs(hand_landmarks.landmark[4].x - hand_landmarks.landmark[9].x) < 0.2:
        return True
    else:
        return False
    
    
def check_thumbs_rx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    nocche=[all_y[i] for i in indici_nocche]

    indici_tips=[8, 12, 16, 20]
    tips=[all_y[i] for i in indici_tips]


    dita_chiuse = all(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )


    if min(thumb_x) > max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.2 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.2 and\
        min(tips)>max(nocche) and\
        dita_chiuse:
        return True
    else:
        return False
    
    
def check_thumbs_sx(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    # new
    thumb_x = all_x[2:5]
    other_x = all_x[5:] #but not 0

    indici_nocche=[5, 9, 13, 17]
    nocche=[all_y[i] for i in indici_nocche]

    indici_tips=[8, 12, 16, 20]
    tips=[all_y[i] for i in indici_tips]
    
    dita_chiuse = all(
        all_y[tip] > all_y[nocca]
        for tip, nocca in zip(indici_tips, indici_nocche)
    )

    if min(thumb_x) < max(other_x) and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[17].y) < 0.2 and\
        abs(hand_landmarks.landmark[4].y - hand_landmarks.landmark[9].y) < 0.2 and\
            dita_chiuse:
        return True
    else:
        return False


def fuuu(hand_landmarks):
    all_y = [lm.y for lm in hand_landmarks.landmark]
    all_x = [lm.x for lm in hand_landmarks.landmark]
    
    all=all_y[0:8]+all_y[13:]
    
    return hand_landmarks.landmark[11].y < min(all)

#### Test di verifica della detection di gesture
- "Scommenta" la seguente cella di codice e eseguila assieme alle precedenti per controllare se le gesture vengono riconosciute correttamente
- Ricorda che la cam si chiude premendo `q` o, in caso non funzionasse, `restartando` il kernel

In [5]:
# #test if gesture are effectively detected - no functions applied


# now = time.time()

# mp_hands = mp.solutions.hands
# mp_draw = mp.solutions.drawing_utils
# hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# # Open webcam
# cap = cv2.VideoCapture(0)  # Change to 1 if using an external webcam

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # Flip the frame horizontally for a mirror effect
#     frame = cv2.flip(frame, 1)
    
#     # Convert to RGB
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
#     # Process the frame and get hand landmarks
#     results = hands.process(rgb_frame)

#     # Draw hand landmarks and print
#     if results.multi_hand_landmarks:
#         for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):
#             # print(f"\nHand {hand_id + 1} detected:")
            
#             for i, lm in enumerate(hand_landmarks.landmark):
#                 # print(f"  Landmark {i}: (X: {lm.x:.4f}, Y: {lm.y:.4f}, Z: {lm.z:.4f})")
#                 # Draw the landmarks on the hand
#                 mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
#             if check_open_hand(hand_landmarks):
#                 # Visualizza sul frame
#                 x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#                 cv2.putText(frame, "MANO APERTA", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                
                
#             elif check_thumbs_up(hand_landmarks):
#                 # Visualizza sul frame
#                 x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#                 cv2.putText(frame, "POLLICE SU'", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    
#             elif check_thumbs_down(hand_landmarks):
#                 # Visualizza sul frame
#                 x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#                 cv2.putText(frame, "POLLICE GIU'", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                
#             elif  check_thumbs_rx(hand_landmarks):
#                 # Visualizza sul frame
#                 x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#                 cv2.putText(frame, "POLLICE DX", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                
#             elif  check_thumbs_sx(hand_landmarks):
#                 # Visualizza sul frame
#                 x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#                 cv2.putText(frame, "POLLICE SX", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                
#             # elif  fuuu(hand_landmarks):
#             #     # Visualizza sul frame
#             #     x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
#             #     cv2.putText(frame, "FANC**", (x, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#             #     # Stampa sul terminale
#             #     print("FANC**")

#     # Display the output
#     cv2.imshow("Hand Tracking Check - no functions", frame)

#     # Exit when 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


#### Codice per controllare la musica tramite gesture
- Esegui i passi iniziali contenuti nel primo markdown
- Ricorda che la cam si chiude premendo `q` o, in caso non funzionasse, `restartando` il kernel
- Ricorda di commentare la cella precendente o di avviare solo quella sottostante

In [6]:
#Er codice

# Variabili globali
last_trigger_time = 0
cooldown = 1 # Tempo di attesa tra i comandi (in secondi)
detection_start_time = 0
is_detecting = False
current_gesture = None
detection_delay = 1.3  # Delay prima di eseguire l'azione dopo il riconoscimento del gesto (in secondi)

# Nuove variabili per tener traccia delle ripetizioni consecutive
consecutive_thumbs_up_count = 0
consecutive_thumbs_down_count = 0
last_gesture = None

# Percorsi dei file audio per i feedback sonori
# Sostituisci questi percorsi con i file audio reali sul tuo sistema

path_start_sound = r"feedbacksounds\start.mp3"
path_completed_sound = r"feedbacksounds\stop.mp3"
# Initialize pygame mixer
pygame.mixer.init()
# Load the sound file
DETECTION_START_SOUND  = pygame.mixer.Sound(path_start_sound)
GESTURE_COMPLETED_SOUND = pygame.mixer.Sound(path_completed_sound)
# Set volume (0.0 to 1.0)
DETECTION_START_SOUND.set_volume(1)
GESTURE_COMPLETED_SOUND.set_volume(1)
# Play the sound
# sound.play()





def handle_gesture(name, action_fn, hand_landmarks, frame):
    global last_trigger_time, detection_start_time, is_detecting, current_gesture
    global consecutive_thumbs_up_count, consecutive_thumbs_down_count, last_gesture
    
    # Se non stiamo già rilevando un gesto, inizia il processo di rilevamento
    if not is_detecting:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        # Riproduci il suono di inizio rilevamento
        DETECTION_START_SOUND.play()
        return
    
    # Se stiamo già rilevando e il gesto è cambiato, resetta il timer
    if current_gesture != name:
        is_detecting = True
        detection_start_time = time.time()
        current_gesture = name
        # Riproduci il suono di inizio rilevamento per il nuovo gesto
        DETECTION_START_SOUND.play()
        return
    
    # Calcola il tempo trascorso dall'inizio del rilevamento
    elapsed_time = time.time() - detection_start_time
    
    # Posizione per visualizzare il nome del gesto e il cerchio
    x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
    
    # Disegna il nome del gesto
    cv2.putText(frame, name.upper(), (x, y-70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Disegna il cerchio di caricamento
    radius = 30
    center = (x, y-20)
    # Disegna il cerchio di sfondo (grigio)
    cv2.circle(frame, center, radius, (100, 100, 100), 3)
    
    # Calcola l'angolo in base al tempo trascorso (da 0 a 360 gradi)
    angle = min(elapsed_time / detection_delay * 360, 360)
    
    # Disegna l'arco di caricamento (verde)
    start_angle = -90  # Inizia dall'alto
    end_angle = start_angle + angle
    
    # Disegna l'arco di caricamento punto per punto
    for i in range(int(start_angle), int(end_angle)):
        rad = math.radians(i)
        x1 = int(center[0] + radius * math.cos(rad))
        y1 = int(center[1] + radius * math.sin(rad))
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), 3)
    
    # Se il tempo di attesa è completato, esegui l'azione
    if elapsed_time >= detection_delay:
        action_fn()
        print(f"{name.upper()} RILEVATO!")
        
        GESTURE_COMPLETED_SOUND.play()  # Riproduci il suono di completamento gesto
        
        
        # Aggiorna i contatori per i gesti consecutivi
        if name == "thumbs_up":
            if last_gesture == "thumbs_up":
                consecutive_thumbs_up_count += 1
            else:
                consecutive_thumbs_up_count = 1
            consecutive_thumbs_down_count = 0
        elif name == "thumbs_down":
            if last_gesture == "thumbs_down":
                consecutive_thumbs_down_count += 1
            else:
                consecutive_thumbs_down_count = 1
            consecutive_thumbs_up_count = 0
        else:
            consecutive_thumbs_up_count = 0
            consecutive_thumbs_down_count = 0
            
        last_gesture = name
        last_trigger_time = time.time()
        is_detecting = False
        current_gesture = None

# Verifica se i file audio esistono o crea file audio di default
def create_default_sounds():
    if not os.path.exists(path_start_sound):
        print(f"File audio di inizio rilevamento non trovato: {path_start_sound}")
        print("Si consiglia di scaricare o creare file audio per i feedback sonori.")
    
    if not os.path.exists(path_completed_sound):
        print(f"File audio di completamento gesto non trovato: {path_completed_sound}")
        print("Si consiglia di scaricare o creare file audio per i feedback sonori.")


def estimate_hand_side(landmarks, handedness):
    WRIST = 0
    INDEX_MCP = 5
    PINKY_MCP = 17

    wrist = np.array([landmarks[WRIST].x, landmarks[WRIST].y, landmarks[WRIST].z])
    index_mcp = np.array([landmarks[INDEX_MCP].x, landmarks[INDEX_MCP].y, landmarks[INDEX_MCP].z])
    pinky_mcp = np.array([landmarks[PINKY_MCP].x, landmarks[PINKY_MCP].y, landmarks[PINKY_MCP].z])

    vec1 = index_mcp - wrist
    vec2 = pinky_mcp - wrist

    normal = np.cross(vec1, vec2)

    # Se la mano è sinistra, invertiamo la normale
    if handedness == 'Left':
        normal = -normal

    if normal[2] > 0:
        return "PALM"
    else:
        return "BACK"


# Inizializzazione di MediaPipe e della webcam
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.7, 
    min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)

# Verifica i file audio all'avvio
create_default_sounds()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # Resetta il flag di rilevamento se è passato troppo tempo senza rilevare lo stesso gesto
    if is_detecting and (time.time() - detection_start_time > detection_delay * 1.5):
        is_detecting = False
        current_gesture = None
    
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Controlla se possiamo saltare il cooldown per i gesti "thumbs_up" e "thumbs_down"
            skip_cooldown = False
            
            if last_gesture == "thumbs_up" and consecutive_thumbs_up_count >= 2:
                if check_thumbs_up(hand_landmarks):
                    skip_cooldown = True
            
            if last_gesture == "thumbs_down" and consecutive_thumbs_down_count >= 2:
                if check_thumbs_down(hand_landmarks):
                    skip_cooldown = True
            
            # Verifica il cooldown o se possiamo saltarlo
            if ((time.time() - last_trigger_time) > cooldown) or skip_cooldown:
                if check_open_hand(hand_landmarks) and estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "PALM" and ((estimate_hand_side(hand_landmarks.landmark, hand_handedness.classification[0].label) == "BACK")==False):
                    handle_gesture("open_hand", play_pause, hand_landmarks, frame)
                elif check_thumbs_up(hand_landmarks):
                    handle_gesture("thumbs_up", vol_up, hand_landmarks, frame)
                elif check_thumbs_rx(hand_landmarks) :
                    handle_gesture("thumbs_rx", next, hand_landmarks, frame)
                elif check_thumbs_sx(hand_landmarks) :
                    handle_gesture("thumbs_sx", prev, hand_landmarks, frame)
                elif check_thumbs_down(hand_landmarks):
                    handle_gesture("thumbs_down", vol_dw, hand_landmarks, frame)
    
    # Visualizza il contatore di utilizzi consecutivi (opzionale)
    if consecutive_thumbs_up_count >= 2:
        cv2.putText(frame, f"Thumbs UP mode: {consecutive_thumbs_up_count}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    if consecutive_thumbs_down_count >= 2:
        cv2.putText(frame, f"Thumbs DOWN mode: {consecutive_thumbs_down_count}", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow("Handle music with gestures + palm ", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[✓] Comando 'pl_pause' eseguito
OPEN_HAND RILEVATO!
[✓] Comando 'pl_next' eseguito
THUMBS_RX RILEVATO!
[✓] Comando 'pl_pause' eseguito
OPEN_HAND RILEVATO!


In [7]:
# # Er codice senza back palm detection


# # Variabili globali
# last_trigger_time = 0
# cooldown = 1
# detection_start_time = 0
# is_detecting = False
# current_gesture = None
# detection_delay = 1.2  # Delay di 1 secondo

# # Nuove variabili per tener traccia delle ripetizioni consecutive
# consecutive_thumbs_up_count = 0
# consecutive_thumbs_down_count = 0
# last_gesture = None

# # Percorsi dei file audio per i feedback sonori
# # Sostituisci questi percorsi con i file audio reali sul tuo sistema

# path_start_sound = r"feedbacksounds\start.mp3"
# path_completed_sound = r"feedbacksounds\stop.mp3"

# # Initialize pygame mixer
# pygame.mixer.init()
# # Load the sound file
# DETECTION_START_SOUND  = pygame.mixer.Sound(path_start_sound)
# GESTURE_COMPLETED_SOUND = pygame.mixer.Sound(path_completed_sound)
# # Set volume (0.0 to 1.0)
# DETECTION_START_SOUND.set_volume(1)
# GESTURE_COMPLETED_SOUND.set_volume(1)
# # Play the sound
# # sound.play()





# def handle_gesture(name, action_fn, hand_landmarks, frame):
#     global last_trigger_time, detection_start_time, is_detecting, current_gesture
#     global consecutive_thumbs_up_count, consecutive_thumbs_down_count, last_gesture
    
#     # Se non stiamo già rilevando un gesto, inizia il processo di rilevamento
#     if not is_detecting:
#         is_detecting = True
#         detection_start_time = time.time()
#         current_gesture = name
#         # Riproduci il suono di inizio rilevamento
#         DETECTION_START_SOUND.play()
#         return
    
#     # Se stiamo già rilevando e il gesto è cambiato, resetta il timer
#     if current_gesture != name:
#         is_detecting = True
#         detection_start_time = time.time()
#         current_gesture = name
#         # Riproduci il suono di inizio rilevamento per il nuovo gesto
#         DETECTION_START_SOUND.play()
#         return
    
#     # Calcola il tempo trascorso dall'inizio del rilevamento
#     elapsed_time = time.time() - detection_start_time
    
#     # Posizione per visualizzare il nome del gesto e il cerchio
#     x, y = int(hand_landmarks.landmark[0].x * frame.shape[1]), int(hand_landmarks.landmark[0].y * frame.shape[0])
    
#     # Disegna il nome del gesto
#     cv2.putText(frame, name.upper(), (x, y-70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
#     # Disegna il cerchio di caricamento
#     radius = 30
#     center = (x, y-20)
#     # Disegna il cerchio di sfondo (grigio)
#     cv2.circle(frame, center, radius, (100, 100, 100), 3)
    
#     # Calcola l'angolo in base al tempo trascorso (da 0 a 360 gradi)
#     angle = min(elapsed_time / detection_delay * 360, 360)
    
#     # Disegna l'arco di caricamento (verde)
#     start_angle = -90  # Inizia dall'alto
#     end_angle = start_angle + angle
    
#     # Disegna l'arco di caricamento punto per punto
#     for i in range(int(start_angle), int(end_angle)):
#         rad = math.radians(i)
#         x1 = int(center[0] + radius * math.cos(rad))
#         y1 = int(center[1] + radius * math.sin(rad))
#         cv2.circle(frame, (x1, y1), 1, (0, 255, 0), 3)
    
#     # Se il tempo di attesa è completato, esegui l'azione
#     if elapsed_time >= detection_delay:
#         action_fn()
#         print(f"{name.upper()} RILEVATO!")
        
#         GESTURE_COMPLETED_SOUND.play()  # Riproduci il suono di completamento gesto
        
        
#         # Aggiorna i contatori per i gesti consecutivi
#         if name == "thumbs_up":
#             if last_gesture == "thumbs_up":
#                 consecutive_thumbs_up_count += 1
#             else:
#                 consecutive_thumbs_up_count = 1
#             consecutive_thumbs_down_count = 0
#         elif name == "thumbs_down":
#             if last_gesture == "thumbs_down":
#                 consecutive_thumbs_down_count += 1
#             else:
#                 consecutive_thumbs_down_count = 1
#             consecutive_thumbs_up_count = 0
#         else:
#             consecutive_thumbs_up_count = 0
#             consecutive_thumbs_down_count = 0
            
#         last_gesture = name
#         last_trigger_time = time.time()
#         is_detecting = False
#         current_gesture = None

# # Verifica se i file audio esistono o crea file audio di default
# def create_default_sounds():
#     if not os.path.exists(path_start_sound):
#         print(f"File audio di inizio rilevamento non trovato: {path_start_sound}")
#         print("Si consiglia di scaricare o creare file audio per i feedback sonori.")
    
#     if not os.path.exists(path_completed_sound):
#         print(f"File audio di completamento gesto non trovato: {path_completed_sound}")
#         print("Si consiglia di scaricare o creare file audio per i feedback sonori.")

# # Inizializzazione di MediaPipe e della webcam
# mp_hands = mp.solutions.hands
# mp_draw = mp.solutions.drawing_utils
# hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
# cap = cv2.VideoCapture(0)

# # Verifica i file audio all'avvio
# create_default_sounds()

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     frame = cv2.flip(frame, 1)
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = hands.process(rgb_frame)
    
#     # Resetta il flag di rilevamento se è passato troppo tempo senza rilevare lo stesso gesto
#     if is_detecting and (time.time() - detection_start_time > detection_delay * 1.5):
#         is_detecting = False
#         current_gesture = None
    
#     if results.multi_hand_landmarks:
#         for hand_landmarks in results.multi_hand_landmarks:
#             mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
#             # Controlla se possiamo saltare il cooldown per i gesti "thumbs_up" e "thumbs_down"
#             skip_cooldown = False
            
#             if last_gesture == "thumbs_up" and consecutive_thumbs_up_count >= 2:
#                 if check_thumbs_up(hand_landmarks):
#                     skip_cooldown = True
            
#             if last_gesture == "thumbs_down" and consecutive_thumbs_down_count >= 2:
#                 if check_thumbs_down(hand_landmarks):
#                     skip_cooldown = True
            
#             # Verifica il cooldown o se possiamo saltarlo
#             if ((time.time() - last_trigger_time) > cooldown) or skip_cooldown:
#                 if check_open_hand(hand_landmarks):
#                     handle_gesture("open_hand", play_pause, hand_landmarks, frame)
#                 elif check_thumbs_up(hand_landmarks):
#                     handle_gesture("thumbs_up", vol_up, hand_landmarks, frame)
#                 elif check_thumbs_rx(hand_landmarks):
#                     handle_gesture("thumbs_rx", next, hand_landmarks, frame)
#                 elif check_thumbs_sx(hand_landmarks):
#                     handle_gesture("thumbs_sx", prev, hand_landmarks, frame)
#                 elif check_thumbs_down(hand_landmarks):
#                     handle_gesture("thumbs_down", vol_dw, hand_landmarks, frame)
    
#     # Visualizza il contatore di utilizzi consecutivi (opzionale)
#     if consecutive_thumbs_up_count >= 2:
#         cv2.putText(frame, f"Thumbs UP mode: {consecutive_thumbs_up_count}", (10, 30), 
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#     if consecutive_thumbs_down_count >= 2:
#         cv2.putText(frame, f"Thumbs DOWN mode: {consecutive_thumbs_down_count}", (10, 60), 
#                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
#     cv2.imshow("Handle music with gestures", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [8]:
# # implementazione di un algoritmo per determinare se la mano è palmo o dorso

# import cv2
# import mediapipe as mp
# import numpy as np

# # Inizializzazione MediaPipe Hands
# mp_hands = mp.solutions.hands
# mp_drawing = mp.solutions.drawing_utils

# hands = mp_hands.Hands(
#     max_num_hands=1,
#     model_complexity=1,
#     min_detection_confidence=0.7,
#     min_tracking_confidence=0.7
# )

# def estimate_hand_side(landmarks, handedness):
#     WRIST = 0
#     INDEX_MCP = 5
#     PINKY_MCP = 17

#     wrist = np.array([landmarks[WRIST].x, landmarks[WRIST].y, landmarks[WRIST].z])
#     index_mcp = np.array([landmarks[INDEX_MCP].x, landmarks[INDEX_MCP].y, landmarks[INDEX_MCP].z])
#     pinky_mcp = np.array([landmarks[PINKY_MCP].x, landmarks[PINKY_MCP].y, landmarks[PINKY_MCP].z])

#     vec1 = index_mcp - wrist
#     vec2 = pinky_mcp - wrist

#     normal = np.cross(vec1, vec2)

#     # Se la mano è sinistra, invertiamo la normale
#     if handedness == 'Left':
#         normal = -normal

#     if normal[2] > 0:
#         return "PALM"
#     else:
#         return "BACK"

# cap = cv2.VideoCapture(0)

# while cap.isOpened():
#     success, frame = cap.read()
#     if not success:
#         print("Ignoring empty frame.")
#         continue

#     frame = cv2.flip(frame, 1)
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = hands.process(frame_rgb)

#     if results.multi_hand_landmarks and results.multi_handedness:
#         for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
#             # Ottieni se la mano è destra o sinistra
#             handedness_label = hand_handedness.classification[0].label

#             # Stima palmo o dorso correttamente
#             hand_side = estimate_hand_side(hand_landmarks.landmark, handedness_label)

#             # Disegna la mano
#             mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

#             # Scrivi l'informazione
#             cv2.putText(frame, f"{handedness_label} Hand - {hand_side}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
#                         1, (0, 255, 0), 2, cv2.LINE_AA)

#     cv2.imshow('Palm vs Back Detection', frame)

#     if cv2.waitKey(5) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()
# hands.close()
